<a href="https://colab.research.google.com/github/vedoly/cadd-smart-systematic-literature-reviews/blob/main/cadd_smart_systematic_literature_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Get Data**

##Download Data

In [1]:
!gdown --id 15CyLhkOFnHIQhqmrjORRjQIgGdEelYHH

Downloading...
From: https://drive.google.com/uc?id=15CyLhkOFnHIQhqmrjORRjQIgGdEelYHH
To: /content/csv-drugdiscov-set.csv
100% 349k/349k [00:00<00:00, 3.02MB/s]


In [2]:
import pandas as pd
df = pd.read_csv('csv-drugdiscov-set.csv')

In [3]:
df.head(1)

,PMID,Title,Authors,Citation,First Author,Journal/Book,Publication Year,Create Date,PMCID,NIHMS ID,DOI
0,30976107,Applications of machine learning in drug disco...,"Vamathevan J, Clark D, Czodrowski P, Dunham I,...",Nat Rev Drug Discov. 2019 Jun;18(6):463-477. d...,Vamathevan J,Nat Rev Drug Discov,2019,2019/04/13,PMC6552674,NIHMS1029624,10.1038/s41573-019-0024-5


In [4]:
df = df[['PMID','Title','Journal/Book','Create Date']]

In [5]:
len(df)

1015

## Get Abstract from PubMed

In [8]:
import requests
import numpy as np

def getAbstract(pmid):
  try:
    page = requests.get("https://pubmed.ncbi.nlm.nih.gov/"+str(pmid) )
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup.find_all(class_='abstract-content selected')[0].p.text.strip()
  except:
    return np.nan

In [9]:
# around 10 min
df['abstract'] = df['PMID'].apply(lambda x: getAbstract(x))
df = df.dropna()

In [26]:
df.head(10)

,PMID,Title,Journal/Book,Create Date,abstract
0,30976107,Applications of machine learning in drug disco...,Nat Rev Drug Discov,2019/04/13,Drug discovery and development pipelines are l...
1,29750902,Machine learning in chemoinformatics and drug ...,Drug Discov Today,2018/05/12,Chemoinformatics is an established discipline ...
2,29887378,Next-Generation Machine Learning for Biologica...,Cell,2018/06/12,"Machine learning, a collection of data-analyti..."
3,31000803,Exploiting machine learning for end-to-end dru...,Nat Mater,2019/04/20,A variety of machine learning methods such as ...
4,29879881,Machine Learning-based Virtual Screening and I...,Curr Pharm Des,2018/06/09,Background:\n \n \n Virtual S...
5,30084866,Recent applications of deep learning and machi...,Brief Bioinform,2018/08/08,The identification of interactions between dru...
6,31905263,Machine Learning in Drug Discovery and Develop...,CPT Pharmacometrics Syst Pharmacol,2020/01/07,"Artificial intelligence, in particular machine..."
7,29603063,Deep Learning for Drug Design: an Artificial I...,AAPS J,2018/04/01,"Over the last decade, deep learning (DL) metho..."
8,31294972,Concepts of Artificial Intelligence for Comput...,Chem Rev,2019/07/12,"Artificial intelligence (AI), and, in particul..."
9,31174387,A Structure-Based Drug Discovery Paradigm,Int J Mol Sci,2019/06/09,Structure-based drug design is becoming an ess...


# **Preprocessing Data**

## Cleaning Abstract Text

In [40]:
test_text = df['abstract'].iloc[0]

In [41]:
test_text

'Drug discovery and development pipelines are long, complex and depend on numerous factors. Machine learning (ML) approaches provide a set of tools that can improve discovery and decision making for well-specified questions with abundant, high-quality data. Opportunities to apply ML occur in all stages of drug discovery. Examples include target validation, identification of prognostic biomarkers and analysis of digital pathology data in clinical trials. Applications have ranged in context and methodology, with some approaches yielding accurate predictions and insights. The challenges of applying ML lie primarily with the lack of interpretability and repeatability of ML-generated results, which may limit their application. In all areas, systematic and comprehensive high-dimensional data still need to be generated. With ongoing efforts to tackle these issues, as well as increasing awareness of the factors needed to validate ML approaches, the application of ML can promote data-driven dec

In [59]:
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
stopwords = nltk.corpus.stopwords.words('english')

def cleanText(text):
  text = text.replace("\n"," ")
  text = text.replace("\t"," ")
  text = text.lower()
  text = re.sub("\s+"," ", text)
  text = re.sub("[^-9A-Za-z ]", "" , text)
  text = nltk.tokenize.word_tokenize(text)
  text = [i for i in text if i not in stopwords]
  text = [nltk.PorterStemmer().stem(word) for word in text]
  text = [nltk.WordNetLemmatizer().lemmatize(word) for word in text]
  return text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [66]:
df['clean_abstract'] = df['abstract'].apply(lambda x:cleanText(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [68]:
df.to_csv('drug_abstract.csv',index=False)

In [ ]:
ps = nltk.PorterStemmer()
text = [nltk.PorterStemmer().stem(word) for word in text]
print(w)